<a href="https://colab.research.google.com/github/user8391/Deep-Learning-/blob/main/Overfitting_Cross_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overfitting - Model Learns too much from the training dataset and deduces noise as hidden patterns in the dataset , leading to good performance with training dataset but often terrible performance with testing dataset .

Methods to avoid Overfitting -
1 . CROSS VALIDATION
2 . Use REGULARIZATION

# Implementing Cross Validation manually using numpy

In [ ]:
# ALL Data = Training set(usually 80%) + Hold out set devset 10%  + Test set 10%
import torch
import torch.nn as nn
import numpy as np
import seaborn as sns

In [ ]:
# Loading dataset
iris = sns.load_dataset('iris') # dataframe

data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(data.shape[0], dtype=torch.long)
labels[iris.species=='versicolor'] = 1
labels[iris.species=='virginica'] = 2

In [ ]:
# Seperating dataset into train and test (no devset here)
'important changes'

# how many training examples
training_proportion = 0.8
training_quantity = int(training_proportion*len(labels) )

# creating bool vector to choose from the dataset
train_test_bool = np.zeros(len(labels) , dtype = bool)

# randomly selecting points from the dataset
randomizing_selection = np.random.choice(range(len(labels)),training_quantity , replace = False)
train_test_bool[randomizing_selection] = True

train_test_bool

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True, False,  True, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True, False,
       False,  True, False,  True,  True, False,  True,  True, False,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,

In [ ]:
model = nn.Sequential(
    nn.Linear(4,12),
    nn.ReLU(),
    nn.Linear(12,12),
    nn.ReLU(),
    nn.Linear(12,3)
)

epochs = 1000
losses = np.zeros(epochs)
current_acc = []
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters() , lr = 0.01)

for epochi in range(epochs):

    'important changes'
    # forward
    yHat = model(data[train_test_bool , :]) # only trains on the instances where the value of train_test_bool is True

    current_acc.append(100*torch.mean((torch.argmax(yHat , axis = 1) == labels[train_test_bool]).float()))

    # losses
    loss = loss_fn(yHat , labels[train_test_bool])
    losses[epochi] = loss

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# final training data forward pass
yHat = model(data[train_test_bool , :]) # for training data
trainingacc = 100*torch.mean((torch.argmax(yHat , axis = 1) == labels[train_test_bool]).float())

# final test accuracy
yHat = model(data[~train_test_bool , :])  # ~ for testing data as '~' negates the Trues
testingacc = 100*torch.mean((torch.argmax(yHat , axis = 1) == labels[~train_test_bool]).float())

print(f'training acc: {trainingacc}')
print(f'testing acc: {testingacc}')


training acc: 99.16666412353516
testing acc: 96.66666412353516


# Cross validation with scikitlearn


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# New
from sklearn.model_selection import train_test_split # function is used for splitting the data into training set and testing set

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')
data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(data.shape[0] , dtype=torch.long)
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2

# the given function splits the data and their labels into training data , labels and testing data , labels
# makes it very easy as compared to building logically from ground up

training_data , testing_data , training_labels , testing_labels = train_test_split(data , labels , test_size=0.2)
# test_size tells the function about the size of the proportion
# or use train size
# train_size tells the function about the size of the proportion

# It also Randomizes the training and the testing data by default without changing the order of features | to disable type shuffle = False

print(training_data.shape)
print(testing_data.shape)
print(training_labels)
print(testing_labels)

torch.Size([120, 4])
torch.Size([30, 4])
tensor([0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 1, 2, 0, 0, 1, 1, 0, 0, 0, 1, 2, 1,
        1, 1, 2, 0, 0, 0, 2, 0, 2, 1, 0, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 1, 0,
        1, 0, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 0, 2, 0, 2, 1, 2, 1, 1, 1, 2, 0,
        1, 1, 0, 1, 2, 2, 0, 2, 0, 1, 1, 1, 1, 2, 1, 0, 2, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 2, 0, 2, 1, 1, 2, 0, 1, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 0, 1, 1, 1])
tensor([0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 2, 2, 1, 0, 2, 2, 1, 1, 1, 1, 2, 1, 1, 0,
        1, 0, 0, 0, 2, 2])


In [ ]:
model = nn.Sequential(
    nn.Linear(4,12),
    nn.ReLU(),
    nn.Linear(12,12),
    nn.ReLU(),
    nn.Linear(12,3)
)

epochs = 1000
losses = np.zeros(epochs)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters() , lr = 0.01)

for epochi in range(epochs):


    # forward
    yHat = model(training_data) # only trains on the instances where the value of train_test_bool is True



    # losses
    loss = loss_fn(yHat , training_labels)
    losses[epochi] = loss

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# final training data forward pass
yHat = model(training_data) # for training data
trainingacc = 100*torch.mean((torch.argmax(yHat , axis = 1) == training_labels).float())

# final test accuracy
outputs = model(testing_data)
testingacc = 100*torch.mean((torch.argmax(outputs , axis = 1) == testing_labels).float())

print(f'training acc: {trainingacc}')
print(f'testing acc: {testingacc}')

/tmp/ipython-input-4009412988.py:24: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:836.)
  losses[epochi] = loss


training acc: 98.33333587646484
testing acc: 93.33333587646484


# Cross validation using Data-Loaders

In [ ]:
# will be used when using with pytorch